In [1]:
#this script creates csv files from data in data/FigS20_simulation 
#plotting for FigS20 was then done in Igor 7, packed experiment file available at figures/FigS20
import os
from pathlib import Path
import sys
sys.path.append(os.path.join(Path(os.getcwd()).parents[0], "functions"))
import warnings
warnings.filterwarnings('ignore')
import slicing_functions as sf
import helper_functions as hfn
import pandas as pd
import matplotlib.pyplot as plt

%config Completer.use_jedi = False

In [2]:
path = Path(os.getcwd()).parents[0]
datapath = os.path.join(path, 'data/FigS20_simulation/')
outpath = os.path.join(datapath, 'figure_data')
hfn.dir_check(outpath)

In [4]:
df_data = pd.read_hdf(os.path.join(datapath, 'results_final.h5'))

In [11]:
df_points = pd.DataFrame ()
df_avg = pd.DataFrame ()
df_std = pd.DataFrame ()
df_var = pd.DataFrame ()

for targetThickness in df_data.thickness.unique():
    expID = str(targetThickness)
    for targetKeep_every in df_data.keep_every.unique():
        
        df_sliced1 = df_data.set_index('thickness').loc[targetThickness].reset_index()
        df_sliced2 = df_sliced1.set_index('keep_every').loc[targetKeep_every].reset_index()

        if 'density_real' not in df_points:
            df_points['density_real']= df_sliced2.packing_density
        
        if 'seed' not in df_points:
            df_points['seed']= df_sliced2.seed
            
        if 'density' not in df_points:
            df_points['density']= df_sliced2.density

        newCol = 'kept'+str(int(targetKeep_every))
        df_points[newCol]= df_sliced2.cells_ratio
       
        #using target density to average across repetitions      
        df_avg = df_points.groupby(['density']).mean()
        df_std = df_points.groupby(['density']).std()
        df_var = df_points.groupby(['density']).var()
        
        #saving to csv
        df_points.to_csv(os.path.join(outpath, str('points_'+expID+'µm_slices.csv')))
        df_avg.to_csv(os.path.join(outpath, str('avg_'+expID+'µm_slices.csv')))
        df_std.to_csv(os.path.join(outpath, str('std_'+expID+'µm_slices.csv')))
        df_var.to_csv(os.path.join(outpath, str('var_'+expID+'µm_slices.csv')))